# 练习29：库和链接

C中的库有两种基本类型：静态、动态

## 静态库

后缀：
* linux: .a
* windows: .lib
    


构建工具：ar 和 ranlib
* 1.写源文件，通过 gcc -c xxx.c 生成目标文件。
* 2.用 ar 归档目标文件，生成静态库。
* 3.配合静态库，写一个使用静态库中函数的头文件。
* 4.使用静态库时，在源码中包含对应的头文件，链接时记得链接自己的库。


这种库可以当做一系 列 .o 对象文件和函数的容器，以及当你构建程序时，可以当做是一个大型 的 .o 文件；


In [6]:
//静态库的构建命令

//my_print.c
void cout(const char * message)
{
    fprintf(stdout, "%s\n", message);
}

//my_math.c
int add(int a, int b)
{
    return a + b;
}

int subtract(int a, int b)
{
    return a - b;
}

//my_lib.h
#ifndef __MY_LIB_H__
#define __MY_LIB_H__

int add(int a, int b);
int subtract(int a, int b);

void cout(const char *);
#endif

//test.c
#include "my_lib.h"

int main(int argc, char *argv[])
{
    int c = add(15, -21);
    cout("I am a func from mylib ...");
    return 0;
}


//编译
gcc -c my_print.c my_math.c
ar crv libmylib.a my_print.o my_math.o
gcc test.c -L. -lmylib




/tmp/tmpaak1jctf.c: In function ‘cout’:
/tmp/tmpaak1jctf.c:6:5: warning: implicit declaration of function ‘fprintf’ [-Wimplicit-function-declaration]
     fprintf(stdout, "%s\n", message);
     ^~~~~~~
/tmp/tmpaak1jctf.c:6:5: warning: incompatible implicit declaration of built-in function ‘fprintf’
/tmp/tmpaak1jctf.c:6:5: note: include ‘<stdio.h>’ or provide a declaration of ‘fprintf’
/tmp/tmpaak1jctf.c:6:13: error: ‘stdout’ undeclared (first use in this function); did you mean ‘cout’?
     fprintf(stdout, "%s\n", message);
             ^~~~~~
             cout
/tmp/tmpaak1jctf.c:6:13: note: each undeclared identifier is reported only once for each function it appears in
/tmp/tmpaak1jctf.c: At top level:
/tmp/tmpaak1jctf.c:31:10: fatal error: my_lib.h: No such file or directory
 #include "my_lib.h"
          ^~~~~~~~~~
compilation terminated.
[C kernel] GCC exited with code 1, the executable will not be executed

## 动态库

后缀：
*  .so （Linux）
 
*  .dll （Windows）
 
 
 构建工具：
  gcc -fPIC -shared
 


这些文件都被构 建好并且放置到指定的地方。当你运行程序时，OS会动态加载这些文件并且“凭 空”链接到你的程序中。



In [ ]:
#include <dlfcn.h>

void *dlopen(const char *filename, int flag);
char *dlerror(void);
void *dlsym(void *handle, const char *symbol);
int dlclose(void *handle);

dlopen以指定模式打开指定的动态连接库文件，并返回一个句柄给调用进程
dlerror返回出现的错误，dlsym通过句柄和连接符名称获取函数名或者变量名
dlclose来卸载打开的库。 

dlopen打开模式如下：
　　RTLD_LAZY 暂缓决定，等有需要时再解出符号 
　　RTLD_NOW 立即决定，返回前解除所有未决定的符号。



//动态库的构建命令
$ cc -c libex29.c -o libex29.o 
$ cc -shared -o libex29.so libex29.o
//编译测试程序
$cc -Wall -g -DNDEBUG ex29.c -ldl -o ex29

## 作者推荐的使用方式

我倾向于对小型或中型项目使用静态的库，因为它们易于使用，并且工作在在更多 操作系统上。我也喜欢将所有代码放入静态库中，之后链接它来执行单元测试，或 者链接到所需的程序中。 

动态库适用于大型系统，它的空间十分有限，或者其中大量程序都使用相同的功 能。这种情况下不应该为每个程序的共同特性静态链接所有代码，而是应该将它放 到动态库中，这样它仅仅会为所有程序加载一份。


## 优缺点

由于能够使用系统调用加载，动态库可以被多种语言的程序调用，而 静态库只能被C及兼容C的程序调用。

# 练习30：自动化测试


C语言的自动化测试比较少，使用的也很少；

这个练习实现了一个C语言的测试框架，这个框架使用了28讲中构建的目录框架，同时也使用了makefile；

```
//目录结构：
bin/   ---- 存放运行程序
build/  ---- makefile存放编译的中间文件
src/   ---- 代码目录
tests/  ---- 测试用例目录
makefile --- make文件
```

## 运行过程

make将src目录文件编译为动态库文件，然后编译测试用例，最后调用脚本执行每个测试用例，输出测试结果；

这个框架将每个测试用例编译为一个小的测试程序；


注意事项：
    
    1.动态库编译的时候，需要将-ldl放到编译指令的文件名后面，暂时还不清楚原因；
    
    2.运行shell脚本的时候，需要注意windows与linux文件换行符的差异，window中未CRLF，linux中只有LF；



In [ ]:
//脚本文件
#!/bin/bash
echo "Running unit tests:"

for i in tests/*_tests
do
    if test -f $i
    then
        if $VALGRIND ./$i 2>> tests/tests.log
        then
            echo $i PASS
        else
            echo "ERROR in test $i: here's tests/tests.log"
            echo "------"
            tail tests/tests.log
            exit 1
        fi
    fi
done

echo ""

//对于每一个测试程序，首先判断是否为一个普通的文件，然后使用valgrind运行测试用例，将测试结果输出到tests.log中，成功打印pass，失败输出log日志；
//test为shell命令，test -f用于测试文件是否为普通的文件

In [ ]:
#include "minunit.h"

char *test_dlopen(int stuff)
{
    return NULL;
}

//。。。。。。。

char *all_tests()
{
    mu_suite_start();

    mu_run_test(test_dlopen);
    mu_run_test(test_functions);
    mu_run_test(test_failures);
    mu_run_test(test_dlclose);

    return NULL;
}

RUN_TESTS(all_tests);

//测试函数具有固定的结构，它们不带任何参数并且返回 char * ，成 功时为 NULL 。这非常重要，因为其他宏用于向测试运行器返回错误信息。
//RUN_TESTS()用于启动一个测试用例，该宏中封装了main（）函数
//mu_suite_start() 用来初始化一个测试用例
//mu_run_test() 启动一个测试项

# 练习31：代码调试

## 调试输出、GDB或Valgrind

大部分时候，使用debug和valgrind就可以解决问题；

使用dbg的场景：
* Valgrind用于捕获所有内存错误。如果Valgrind中含有错误或Valgrind会严重拖 慢程序，我会使用gdb。 
* 调试输出用于诊断或修复有关逻辑或使用上的缺陷。在你使用Valgrind之前， 这些共计90%的缺陷。 
* 使用gdb解决剩下的“谜之bug”，或如要收集信息的紧急情况。如果Valgrind不 起作用，并且我不能打印出所需信息，我就会使用gdb开始四处搜索。这里我 仅仅使用gdb来收集信息。一旦我弄清发生了什么，我会回来编程单元测试来 引发缺陷，之后编程打印语句来查找原因。

调用gdb编译需要在cc后面加 -g参数再加-o；

## gdb的常用命令

```
[root@redhat home]#gdb 调试文件：启动gdb

(gdb) l ：（字母l）从第一行开始列出源码

(gdb) break n :在第n行处设置断点

(gdb) break func：在函数func()的入口处设置断点

(gdb) info break： 查看断点信息

(gdb) r：运行程序

(gdb) n：单步执行

(gdb) c：继续运行

(gdb) p 变量 ：打印变量的值

(gdb) bt：查看函数堆栈

(gdb) finish：退出函数

(gdb) shell 命令行：执行shell命令行

(gdb) set args 参数:指定运行时的参数

(gdb) show args：查看设置好的参数

(gdb) show paths:查看程序运行路径；

           set environment varname [=value] 设置环境变量。如：set env USER=hchen；

            show environment [varname] 查看环境变量；

(gdb) cd 相当于shell的cd;

(gdb)pwd ：显示当前所在目录

(gdb)info program： 来查看程序的是否在运行，进程号，被暂停的原因。

(gdb)clear 行号n：清除第n行的断点

(gdb)delete 断点号n：删除第n个断点

(gdb)disable 断点号n：暂停第n个断点

(gdb)enable 断点号n：开启第n个断点

(gdb)step：单步调试如果有函数调用，则进入函数；与命令n不同，n是不进入调用的函数的
```


## 附加到进程

gdb 最实用的功能就是附加到运行中的程序，并且就地调试它的能力。

```
$ ps ax | grep ex31 
10026 s000 S+ 0:00.11 ./ex31 
10036 s001 R+ 0:00.00 grep ex31 

$ gdb ./ex31 10026
//gdb 打印出了一堆关于协议的信息，接着它读取了所有东西
//程序停在当前点，然后可以执行命令
```


# 练习32：双向链表

数据结构：具有特性模型的数据组织方法


## 双链表

示例：
![示例](https://aliyun-lc-upload.oss-cn-hangzhou.aliyuncs.com/aliyun-lc-upload/uploads/2018/04/17/screen-shot-2018-04-17-at-161130.png)


链表结构体：
```
typedef struct ListNode { 
	struct ListNode *next; 
	struct ListNode *prev; 
	void *value; 
} ListNode; 

typedef struct List { 
	int count; 
	ListNode *first; 
	ListNode *last; 
} List; 

```

## 链表操作

### 添加操作

如果我们想在现有的结点 prev 之后插入一个新的结点 cur，我们可以将此过程分为两个步骤：

1.链接 cur 与 prev 和 next，其中 next 是 prev 原始的下一个节点；
![添加链表1](https://aliyun-lc-upload.oss-cn-hangzhou.aliyuncs.com/aliyun-lc-upload/uploads/2018/04/28/screen-shot-2018-04-28-at-173045.png)

2.用 cur 重新链接 prev 和 next。
![添加链表2](https://aliyun-lc-upload.oss-cn-hangzhou.aliyuncs.com/aliyun-lc-upload/uploads/2018/04/29/screen-shot-2018-04-28-at-173055.png)

添加操作的时间和空间复杂度都是 O(1)

### 删除操作
如果我们想从双链表中删除一个现有的结点 cur，我们可以简单地将它的前一个结点 prev 与下一个结点 next 链接起来。

与单链表不同，使用“prev”字段可以很容易地在常量时间内获得前一个结点。因为我们不再需要遍历链表来获取前一个结点，所以时间和空间复杂度都是O(1)。

In [ ]:
//操作汇总
typedef struct ListNode { 
	struct ListNode *next; 
	struct ListNode *prev; 
	void *value; 
} ListNode; 

typedef struct List { 
	int count; 
	ListNode *first; 
	ListNode *last; 
} List; 

List *List_create(); 
void List_destroy(List *list); 
void List_clear(List *list); 
void List_clear_destroy(List *list); 

#define List_count(A) ((A)->count) 
#define List_first(A) ((A)->first != NULL ? (A)->first->value : NULL) 
#define List_last(A) ((A)->last != NULL ? (A)->last->value : NULL)

void List_push(List *list, void *value); 
void *List_pop(List *list); 
void List_unshift(List *list, void *value); 
void *List_shift(List *list); 
void *List_remove(List *list, ListNode *node); 

#define LIST_FOREACH(L, S, M, V)    \
            ListNode *_node = NULL; \ 
            ListNode *V = NULL;     \
            for(V = _node = L->S; _node != NULL; V = _node = _node->M) 